# <center> Ch 8 PEFT&医疗领域模型微调实践 </center>

# 1. 项目背景

中医是中华医疗领域的宝贵财富，有着几千年的历史，治病救人一直发挥着重要作用。但进入现代社会，中医的发展遇到了一些实际困难：

1. **学习难度大**：  
   中医的经典书籍，比如《黄帝内经》《伤寒杂病论》，内容深奥，很多知识是靠经验总结出来的，没有现代医学那么系统。年轻人学起来觉得很吃力，很多宝贵的知识没能被很好地传承下来。

2. **数字化不足**：  
   很多地方的中医医案和处方还停留在纸质或者手写记录的阶段，这些宝贵的内容很难被整理和分析。相较于西医的数字化进程，中医在数据挖掘和利用上还比较落后。

3. **实用性受限**：  
   现在很多人对中医并不了解，觉得中医是“玄学”或者“养生方法”，对它的实际作用有误解。中医的知识如果能更贴近普通人的需求，可能会让更多人受益。

通过现在的人工智能技术，比如大语言模型，我们可以试着将这些困难转化为机会，让中医知识更加清晰、易懂，也让它的应用更加普及和智能化。


## 1.1 项目意义

#### 1. 帮助中医更好地传承
中医理论复杂，经典文献又很难懂，而通过微调一个中医大模型，可以把这些晦涩的知识用简单的语言表达出来，帮助学生和普通人理解，比如：
- “脾虚”到底是什么意思？
- “湿热”和“寒湿”有什么区别？  

这种方式可以降低中医知识的学习门槛，尤其是帮助年轻一代更好地接触和掌握中医。

#### 2. 给医生提供工具
医生看病需要结合病人的各种症状来判断病因、开方调理。微调后的模型可以作为医生的工具，比如：
- 整理和总结历史医案，让医生更容易参考类似的病情处理方案。
- 提供初步的诊断建议，比如病人描述“舌苔黄腻、口苦”，模型可以提示可能是湿热症。

虽然它不能替代医生，但可以减少医生处理大量信息的负担，提高效率。

#### 3. 普通人的健康帮手
很多人平时有小病小痛或者养生需求，但又不想每次都去医院。如果有一个中医模型，普通人可以随时提问，比如：
- “我最近老是疲惫无力，是不是气虚？该怎么调理？”  
- “夏天湿气重，有什么简单的方法排湿？”  

模型可以根据中医的理论，提供一些基础的调理建议，帮助大家更好地关注自己的身体状态。

#### 4. 为科研提供支持
通过模型对中医医案和数据的分析，可以发现一些潜在的规律，比如：
- 哪些药材组合在某些疾病中效果更好？
- 某种病症的治疗有没有新的突破点？  

这些分析可以帮助科研人员更快找到有价值的研究方向。

## 1.2 项目前景

#### **1. 医疗辅助**
微调后的中医模型可以成为医生的“助手”，帮助医生管理医案、整理思路、提升诊疗效率。尤其是在基层医疗资源不足的地方，模型的应用可能会更加实用。

#### **2. 健康管理**
对于普通人来说，模型可以提供日常养生建议和简单的健康调理方案，让大家更容易理解中医里的“治未病”理念。

#### **3. 数据整理和分析**
中医的历史数据很多，但利用率不高。通过模型的训练和挖掘，可以把这些数据变成更有价值的内容，比如推荐哪些药方更适合特定的病症。

#### **4. 推动中医现代化**
中医的理论很多时候难以用现代医学语言解释清楚，模型可以作为一个桥梁，帮助中医和现代医学更好地结合。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241226115631471.png" width=100%></div>

# 2. 模型验证

模型下载已经多次讲解，本次不再赘述。

当前市面模型较多，并且启动方式也不完全相同，以下简单展示几种供大家参考：

## 2.1 多卡验证

In [ ]:
import torch
# 导入PyTorch库，PyTorch是一个用于深度学习的框架。它提供了很多函数和工具来构建、训练和推理神经网络。
from transformers import AutoModelForCausalLM, AutoTokenizer
# AutoModelForCausalLM 是 transformers 库中用于加载和使用语言模型（如GPT、BERT、LLaMA等）的工具。CausalLM 表示因果语言模型，它通常用于生成式任务，比如自动生成文本。
# AutoTokenizer 用于加载分词器（tokenizer）。分词器负责将文本转化为模型可以理解的数字格式（即token化），并可以将模型的输出数字转换回文本。
import os

# 设置使用的显卡（0, 1, 2表示使用第1、2和3号显卡）
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 

# 是你要加载的模型的路径，或者是模型的名称。在这个例子中，我们指定了一个本地路径，指向一个Qwen模型。如果你使用的是其他模型，比如LLaMA模型，你需要把路径更改为相应的路径。
# model_path = "/home/data/muyan/model/AI-ModelScope/Llama-3.2-1B-Instruct"  
model_path = "/home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct"  

# 加载指定路径下的模型分词器。分词器负责将输入的文本转化为数字形式，供模型处理。这里使用了from_pretrained方法，它会根据你指定的路径自动加载对应的预训练分词器。
tokenizer = AutoTokenizer.from_pretrained(model_path)


# 加载大语言模型，设置使用自动数据类型和设备映射
# `torch_dtype="auto"` 会根据你的硬件自动选择合适的类型（比如使用 float16 以减少显存占用）
# `device_map="auto"` 会根据可用的 GPU 自动选择合适的设备来加载模型。
# 如果机器上有多张显卡，它会自动分配到不同显卡上。你不需要显式地选择。
model = AutoModelForCausalLM.from_pretrained(
    model_path,         
    torch_dtype="auto", 
    device_map="auto" 
)

# 设置pad_token为eos_token（如果没有指定Llama系列模型会报错，同期也要设置长度自动补齐 Qwen2.5不用设置也会自动识别）
# The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
# Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

# 是分词器的填充符号。大多数语言模型需要对输入文本进行填充，使得不同长度的输入可以批量处理（每个输入都填充到相同的长度）。
if tokenizer.pad_token is None:
    # 如果分词器没有指定填充符号（tokenizer.pad_token is None），我们将填充符号设置为 eos_token（即序列结束符）。这是为了防止因缺少填充符号而导致模型错误。
    tokenizer.pad_token = tokenizer.eos_token  

# 是你想要输入给模型的文本。
input_text = "你好，你是谁！"

# input_text 会被分词器转换为模型需要的格式（即 token IDs）。
# return_tensors="pt"：表示返回PyTorch格式的张量，这样可以直接送入模型进行处理。
# padding=True：自动将输入文本填充到相同的长度。
# truncation=True：如果输入文本太长，自动截断为最大长度。
# .to(device)：将处理后的输入张量移动到指定显卡上。
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(model.device)

# 手动设置 attention_mask 和 pad_token_id
# attention_mask 是一个指示哪些位置是填充的二进制掩码。模型在计算注意力时会忽略这些填充位置。inputs["attention_mask"] 将自动包含在之前的分词器处理步骤中，但我们显式地将其移动到正确的设备。
inputs["attention_mask"] = inputs["attention_mask"].to(model.device)
# pad_token_id 是填充符号的ID。我们将其设置为分词器的填充符号ID，以确保模型知道如何处理填充的部分。
inputs["pad_token_id"] = tokenizer.pad_token_id


# 进行推理
# model.generate() 是生成式模型的推理方法。它将根据输入的 input_ids（文本的token IDs）生成输出。
# inputs["input_ids"]：模型需要的输入ID。
# attention_mask：告知模型哪些位置是有效的，哪些是填充的。
# pad_token_id：填充符号的ID，告诉模型哪些部分是填充。
# max_length=50：限制生成的最大长度为50个token。
outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], pad_token_id=inputs["pad_token_id"], max_length=50)

# 个人兴趣做验证使用对比llama模型以及qwen系列模型区别
#==============================
# 原始输入的长度
original_length = len(tokenizer.tokenize(input_text))
# 填充后的长度
padded_length = len(inputs["input_ids"][0])
# 填充的数量
padding_count = padded_length - original_length

print(f"tokenizer.pad_token：{tokenizer.pad_token}")
print(f"tokenizer.eos_token：{tokenizer.eos_token}")
print(f"原始输入长度：{original_length}")
print(f"填充后的长度：{padded_length}")
print(f"填充的占位符数量：{padding_count}")
print("=="*100)
#==============================

# outputs[0] 是生成的第一个序列，可能包含多个生成序列，但这里只取第一个。
# tokenizer.decode(outputs[0], skip_special_tokens=True)：将生成的token IDs转换回文本，并跳过特殊符号（如[PAD]、[SEP]等）。
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(outputs)

tokenizer.pad_token：<|endoftext|>
tokenizer.eos_token：<|im_end|>
原始输入长度：5
填充后的长度：5
填充的占位符数量：0
你好，你是谁！
我是来自阿里云的大规模语言模型，我叫通义千问。有什么我可以帮助你的吗？（10分） 深度学习、自然语言处理、机器翻译、图像识别等技术
tensor([[108386,   3837, 105043, 100165,   6313,    198, 104198, 101919, 102661,
          99718, 104197, 100176, 102064, 104949,   3837,  35946,  99882,  31935,
          64559,  99320,  56007,   1773, 104139, 109944, 100364, 103929, 101037,
          11319,   9909,     16,     15,  17177,   7552,   6567,    115,    109,
          26381, 100134,   5373,  99795, 102064,  54542,   5373, 102182, 105395,
           5373, 107553, 102450,  49567,  99361]], device='cuda:0')


也可以参考官方代码：

In [2]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
# import os
# 控制加载指定显卡(可选)
# # 设置使用的显卡（0, 1, 2表示使用第1、2和3号显卡）
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 


# 定义模型名称，使用 Qwen2.5-7B-Instruct 版本
# model_name = "你的模型存储路径"
model_name = "/home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct"

# 加载预训练的语言模型（因模型较大，指定使用合适的设备和数据类型）
model = AutoModelForCausalLM.from_pretrained(
    model_name,        # 模型的名称或路径
    torch_dtype="auto",  # 自动选择合适的 PyTorch 数据类型（如 float16 或 float32）
    device_map="auto"    # 自动选择设备（如 GPU 或 CPU），如果有多个 GPU 则分配到不同设备
)

# 加载与模型对应的分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 设置输入的提示文本
prompt = "简单介绍一下大型语言模型。"

# 创建消息列表，模拟一个对话场景
messages = [
    {"role": "system", "content": "你是Qwen，阿里云打造的助手"},
    {"role": "user", "content": prompt}
]

# 使用分词器将消息转化为模型所需的格式（并加入生成提示）
text = tokenizer.apply_chat_template(
    messages,              # 消息列表，包含用户和系统的对话
    tokenize=False,        # 不进行标记化，保留原始文本
    add_generation_prompt=True  # 添加生成提示符，通常是为了指引模型如何生成文本
)

# 使用分词器将输入文本编码为模型所需的张量格式，并将其转移到正确的设备（如 GPU）
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# 使用模型生成输出文本，指定最大生成长度（新生成的最大 token 数）
generated_ids = model.generate(
    **model_inputs,      # 将模型输入传递给模型
    max_new_tokens=512   # 限制生成的最大 token 数为 512
)

# 修剪生成的 id，确保生成部分不包含输入的部分
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# 解码生成的 token id 为可读的文本，并去除特殊标记
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# 打印最终生成的回答
print(response)


我是Qwen，由阿里云自主研发的大规模预训练语言模型。我的设计目的是帮助用户理解和生成自然语言，包括但不限于回答问题、撰写文档、创作文章等。我能够理解上下文，并根据这些信息生成具有丰富想象力和深度的理解与表达。此外，我还具备处理复杂任务的能力，如对话管理、创意写作以及数据挖掘等领域。通过大量的预训练和持续的学习，我可以不断优化自己的性能，以更好地服务于用户的需求。


## 界面验证

使用官方程序：注意更改IP地址以及对应的模型加载路径

文件名称为web_demo.py 对应启动命令 python web_demo.py

```python 

"""A simple web interactive chat demo based on gradio."""

from argparse import ArgumentParser
from threading import Thread

import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

# 可以直接启动后续微调完合并后的模型
# DEFAULT_CKPT_PATH = "/home/data/muyan/code/merge_model/1224/24_23_0.5B_200"
# 直接加载下载模型路径
DEFAULT_CKPT_PATH = "/home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct"


def _get_args():
    parser = ArgumentParser(description="Qwen2.5-Instruct web chat demo.")
    parser.add_argument(
        "-c",
        "--checkpoint-path",
        type=str,
        default=DEFAULT_CKPT_PATH,
        help="Checkpoint name or path, default to %(default)r",
    )
    parser.add_argument(
        "--cpu-only", action="store_true", help="Run demo with CPU only"
    )

    parser.add_argument(
        "--share",
        action="store_true",
        default=False,
        help="Create a publicly shareable link for the interface.",
    )
    parser.add_argument(
        "--inbrowser",
        action="store_true",
        default=False,
        help="Automatically launch the interface in a new tab on the default browser.",
    )
    parser.add_argument( 
        "--server-port", type=int, default=8000, help="Demo server port."
    )
    parser.add_argument( # IP地址大家可以自行修改，可以使用127.0.0.1  
        "--server-name", type=str, default="0.0.0.0", help="Demo server name."
    )

    args = parser.parse_args()
    return args


def _load_model_tokenizer(args):
    tokenizer = AutoTokenizer.from_pretrained(
        args.checkpoint_path,
        resume_download=True,
    )

    if args.cpu_only:
        device_map = "cpu"
    else:
        device_map = "auto"

    model = AutoModelForCausalLM.from_pretrained(
        args.checkpoint_path,
        torch_dtype="auto",
        device_map=device_map,
        resume_download=True,
    ).eval()
    model.generation_config.max_new_tokens = 2048  # For chat.

    return model, tokenizer


def _chat_stream(model, tokenizer, query, history):
    conversation = []
    for query_h, response_h in history:
        conversation.append({"role": "user", "content": query_h})
        conversation.append({"role": "assistant", "content": response_h})
    conversation.append({"role": "user", "content": query})
    input_text = tokenizer.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        tokenize=False,
    )
    inputs = tokenizer([input_text], return_tensors="pt").to(model.device)
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer, skip_prompt=True, timeout=60.0, skip_special_tokens=True
    )
    generation_kwargs = {
        **inputs,
        "streamer": streamer,
    }
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    for new_text in streamer:
        yield new_text


def _gc():
    import gc

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def _launch_demo(args, model, tokenizer):
    def predict(_query, _chatbot, _task_history):
        print(f"User: {_query}")
        _chatbot.append((_query, ""))
        full_response = ""
        response = ""
        for new_text in _chat_stream(model, tokenizer, _query, history=_task_history):
            response += new_text
            _chatbot[-1] = (_query, response)

            yield _chatbot
            full_response = response

        print(f"History: {_task_history}")
        _task_history.append((_query, full_response))
        print(f"Qwen: {full_response}")

    def regenerate(_chatbot, _task_history):
        if not _task_history:
            yield _chatbot
            return
        item = _task_history.pop(-1)
        _chatbot.pop(-1)
        yield from predict(item[0], _chatbot, _task_history)

    def reset_user_input():
        return gr.update(value="")

    def reset_state(_chatbot, _task_history):
        _task_history.clear()
        _chatbot.clear()
        _gc()
        return _chatbot

    with gr.Blocks() as demo:
        gr.Markdown("""\
<p align="center"><img src="https://qianwen-res.oss-accelerate-overseas.aliyuncs.com/assets/logo/qwen2.5_logo.png" style="height: 120px"/><p>""")
        gr.Markdown(
            """\
<center><font size=3>This WebUI is based on Qwen2.5-Instruct, developed by Alibaba Cloud. \
(本WebUI基于Qwen2.5-Instruct打造，实现聊天机器人功能。)</center>"""
        )
        gr.Markdown("""\
<center><font size=4>
Qwen2.5-7B-Instruct <a href="https://modelscope.cn/models/qwen/Qwen2.5-7B-Instruct/summary">🤖 </a> | 
<a href="https://huggingface.co/Qwen/Qwen2.5-7B-Instruct">🤗</a>&nbsp ｜ 
Qwen2.5-32B-Instruct <a href="https://modelscope.cn/models/qwen/Qwen2.5-32B-Instruct/summary">🤖 </a> | 
<a href="https://huggingface.co/Qwen/Qwen2.5-32B-Instruct">🤗</a>&nbsp ｜ 
Qwen2.5-72B-Instruct <a href="https://modelscope.cn/models/qwen/Qwen2.5-72B-Instruct/summary">🤖 </a> | 
<a href="https://huggingface.co/Qwen/Qwen2.5-72B-Instruct">🤗</a>&nbsp ｜ 
&nbsp<a href="https://github.com/QwenLM/Qwen2.5">Github</a></center>""")

        chatbot = gr.Chatbot(label="Qwen", elem_classes="control-height")
        query = gr.Textbox(lines=2, label="Input")
        task_history = gr.State([])

        with gr.Row():
            empty_btn = gr.Button("🧹 Clear History (清除历史)")
            submit_btn = gr.Button("🚀 Submit (发送)")
            regen_btn = gr.Button("🤔️ Regenerate (重试)")

        submit_btn.click(
            predict, [query, chatbot, task_history], [chatbot], show_progress=True
        )
        submit_btn.click(reset_user_input, [], [query])
        empty_btn.click(
            reset_state, [chatbot, task_history], outputs=[chatbot], show_progress=True
        )
        regen_btn.click(
            regenerate, [chatbot, task_history], [chatbot], show_progress=True
        )

        gr.Markdown("""\
<font size=2>Note: This demo is governed by the original license of Qwen2.5. \
We strongly advise users not to knowingly generate or allow others to knowingly generate harmful content, \
including hate speech, violence, pornography, deception, etc. \
(注：本演示受Qwen2.5的许可协议限制。我们强烈建议，用户不应传播及不应允许他人传播以下内容，\
包括但不限于仇恨言论、暴力、色情、欺诈相关的有害信息。)""")

    demo.queue().launch(
        share=args.share,
        inbrowser=args.inbrowser,
        server_port=args.server_port,
        server_name=args.server_name,
    )


def main():
    args = _get_args()

    model, tokenizer = _load_model_tokenizer(args)

    _launch_demo(args, model, tokenizer)


if __name__ == "__main__":
    main()
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225172606666.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225172708897.png" width=100%></div>

后端日志显示

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225175718184.png" width=100%></div>

# 3. PEFT介绍

## 3.1 Hugging Face 平台简介

Hugging Face 是一家专注于 **人工智能（AI）技术开发和应用** 的公司。它通过开源工具、社区支持和云服务，构建了一个完整的生态系统，涵盖了自然语言处理（NLP）、计算机视觉（CV）等领域。Hugging Face 的工具和平台在学术研究和工业界中都非常流行。

### Hugging Face 的主要组成部分

#### **1. Model Hub（模型中心）**
Hugging Face 提供了一个全球最大的 **模型共享平台**，被称为 **Model Hub**。它是 Hugging Face 生态系统的核心。
- **功能**：
  - 用户可以上传和下载数以千计的预训练模型（如 GPT-2、BERT、LLaMA 等）。
  - 支持多个框架，包括 **PyTorch**、**TensorFlow** 和 **JAX**。
  - 模型可以直接用于推理、微调和分布式训练。


#### 2. 数据集生态（Datasets Hub）
Hugging Face 提供了一个类似 Model Hub 的 **数据集平台**，称为 **Datasets Hub**。
- **功能**：
  - 用户可以访问超过 10,000 个标准化数据集，用于文本分类、翻译、问答等任务。
  - 数据集可以与模型无缝结合，支持自动分词和批处理。

#### 3. Transformers（核心工具库）
- **Transformers** 是 Hugging Face 的核心库，支持加载和训练预训练的 Transformer 模型。
- **功能**：
  - 加载预训练模型（如 GPT、BERT 等）。
  - 进行任务微调（如文本生成、分类、翻译等）。
  - 推理和评估模型性能。

#### 4. Tokenizers（分词工具库）
- 用于将文本处理成模型可以理解的输入格式（tokens）。
- 支持多种分词算法（如 WordPiece、BPE 等）。
- 高效且支持多线程分词，适合处理大规模数据。

#### 5. Accelerate
- 一个轻量级库，简化分布式训练和推理。
- 适合需要在多 GPU 或 TPU 上运行的项目。

#### 6. PEFT
- Hugging Face 提供的一个库，用于参数高效微调。
- 通过只微调模型的一小部分参数来适配下游任务，从而大幅降低计算资源和存储需求。

**假设你要训练一个翻译模型（从英文翻译成中文），这个过程可以类比为完成一个 “翻译任务”：**

1. **Transformers 是你的翻译工具**：
   - 它像是一本全面的翻译字典，提供了多种语言和预训练知识。

2. **Datasets 提供了练习材料**：
   - 数据集就像一堆翻译练习题，帮助模型掌握翻译能力。

3. **Tokenizers 是你的助手**：
   - Tokenizers 把每句话切分成单词或词组，就像助手帮你分解每句复杂的句子。

4. **Accelerate 是训练加速器**：
   - 它相当于为你提供了一批助手（GPU），让你能更快完成所有翻译练习。

5. **PEFT 是资源受限时的优化策略**：
   - 如果硬件资源不足，全量微调太耗费时间和存储空间，你可以用 PEFT，只优化翻译中特定的句式或用词规则（调整少量参数），以便快速提升效果。


## 3.2 PEFT详解



#### PEFT 是什么？
PEFT 是一种 **参数高效微调技术**，旨在解决微调大型预训练模型时的高资源需求问题。传统的全量微调会调整模型的所有参数，这对存储、计算能力提出了很高的要求。而 PEFT 通过只更新少量参数，实现高效且经济的微调。

#### **PEFT 的主要方法**
1. **LoRA（Low-Rank Adaptation）**
   - 通过引入低秩矩阵，在不改变原始模型参数的基础上，微调少量的参数矩阵。
   - **特点**：适合硬件受限的场景。

2. **Prefix Tuning**
   - 在模型的输入序列中添加一组可训练的前缀向量，以适应下游任务。
   - **特点**：模型参数完全冻结，只调整前缀。

3. **P-Tuning v2**
   - 在模型的每一层插入少量的提示向量，通过这些向量调整模型的行为。
   - **特点**：高效且适用于复杂任务。

4. **Prompt Tuning**
   - 直接优化输入的提示（Prompt），使模型生成符合需求的结果。


**把 Hugging Face 平台比作一家高级定制服装店：**
- **Transformers** 是仓库里的通用服装，已经有了不同尺码和款式的衣服。
- **PEFT** 是裁缝，它通过简单调整（如加点饰品或修改部分细节）就能让衣服更贴合你的需求。
  - **LoRA**：像是加了一个简单的胸针（低秩矩阵），让衣服更适合正式场合。
  - **Prefix Tuning**：像是在衣服上添加一条独特的腰带（前缀向量）。
  - **Prompt Tuning**：像是给衣服加个标签（提示语），说明它适合的场景。

最终，PEFT 的目标是以最小的修改让模型（服装）在新的任务（场合）中表现更好！

#### Hugging Face 和 PEFT 的关系

1. **Hugging Face 是生态平台**：
   - 提供了模型（Transformers）、数据（Datasets）、工具（Accelerate 和 Tokenizers）以及微调技术（PEFT）的一站式解决方案。
   - PEFT 是 Hugging Face 生态中的一部分，专注于高效微调。

2. **PEFT 基于 Transformers 构建**：
   - PEFT 的所有方法都依赖于 Transformers 库提供的模型加载和推理功能。
   - 它通过与 Transformers 集成，提供更加经济的微调方式。

### 之前用到的安装包对比



| **库**         | **作用**                                                                 | **关系**                                                                                  | **区别**                                                                 |
|----------------|--------------------------------------------------------------------------|------------------------------------------------------------------------------------------|--------------------------------------------------------------------------|
| **Transformers** | 提供预训练模型和基础操作（如加载、微调和推理）。                              | 是所有操作的基础，其他库都依赖于 Transformers 提供的模型和功能。                                | 关注模型加载和基础训练，不涉及高效微调或强化学习。                                  |
| **PEFT**        | 通过高效微调方法只调整少量参数来适配下游任务，降低资源需求。                     | 基于 Transformers，微调时调用其模型和数据。                                               | 专注于减少训练资源和存储，适合多任务场景或硬件受限的环境。                          |
| **TRL**         | 使用强化学习优化模型性能，尤其是结合人类反馈的 RLHF 方法。                      | 也依赖于 Transformers 提供的模型，但专注于强化学习优化流程。                                | 专注于通过奖励函数和强化学习算法（如 PPO）提升模型性能，与 PEFT 的目标互补。         |


**将整个微调过程比作一个厨师（模型）学习烹饪新菜品：**

1. **Transformers**：
   - 相当于厨师的基础技能和食谱。例如，GPT 是厨师的基础能力，它已经掌握了很多通用菜品的做法。
   - Transformers 库提供了这些技能和食谱。

2. **PEFT**：
   - 相当于厨师学习新菜品时，不需要重新学习所有技能，只需学会某些特殊技巧或加入一些新调料。
   - PEFT 方法通过只调整少量参数（比如特定的调料配方）来让厨师快速适应新的菜品需求。

3. **TRL**：
   - 相当于通过顾客的反馈优化厨师的烹饪技巧。例如，顾客喜欢菜品的某种味道，厨师通过奖励机制（人类反馈）不断优化自己的做菜方式。
   - TRL 是强化学习，教厨师更好地满足顾客的偏好。

**最终结果**：
- Transformers 提供了厨师的基础能力。
- PEFT 帮助厨师快速掌握新菜品的配方。
- TRL 通过顾客反馈让厨师进一步提升菜品质量。


# 4. 模型微调

- **step1 数据准备**  

选择数据源：

法律数据：https://www.modelscope.cn/datasets?page=1&query=%E6%B3%95%E5%BE%8B&sort=downloads

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224111444755.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224111546546.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224111616135.png" width=100%></div>

中医数据： https://www.modelscope.cn/datasets/alexhuangguo/chinese-medical/files

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224111919438.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224111949666.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224112022652.png" width=100%></div>

金融数据：https://github.com/A-Rain/BDCI2019-Negative_Finance_Info_Judge/tree/master/data

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224114708265.png" width=100%></div>

金融数据合集链接：https://github.com/icoxfog417/awesome-financial-nlp

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224114843768.png" width=100%></div>

译文介绍

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224114902082.png" width=100%></div>

法律类：罪名法务名词及分类模型

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224213153151.png" width=100%></div>

对联数据：https://github.com/wb14123/couplet-dataset?tab=readme-ov-file

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224115528790.png" width=100%></div>

本次使用的是中医问诊数据：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224112143414.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224112209333.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224213957524.png" width=100%></div>

可以查看dataset_infos.json 查看数据集信息，也可以直接使用数据卡片查看信息后直接上传服务器进行数据校验。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224190009822.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224190057870.png" width=100%></div>

开始进行数据处理(当前数据处理格式并不唯一，当前数据格式方便大家理解，后续可以进行自定义处理。)
```python
# 注意：当前只取1000条数据，并且更改了"instruction"、"input"相互之间的对应关系，方便后续统一识别
# 当前文件命名为convert_json.py
import json
import argparse

def convert_json_format(input_file, output_file, num_entries=1000):
    """
    读取输入JSON文件，将每条记录的字段重新排列，并写入输出JSON文件。
    
    :param input_file: 输入JSON文件的路径
    :param output_file: 输出JSON文件的路径
    :param num_entries: 要处理的记录数量
    """
    # 打开并读取输入文件的所有行
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    converted_data = []
    # 遍历每一行数据，并进行字段重新排列
    for line in lines[:num_entries]:
        entry = json.loads(line)
        new_entry = {
            "instruction": entry["input"],  # 将原input字段的内容移到instruction字段
            "input": entry["instruction"],  # 将原instruction字段的内容移到input字段
            "output": entry["output"]       # 保持output字段不变
        }
        converted_data.append(new_entry)

    # 打开输出文件并逐条写入转换后的数据
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(converted_data, outfile, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    # 创建ArgumentParser对象以解析命令行参数
    parser = argparse.ArgumentParser(description="Convert JSON format of medical QA data.")
    
    # 添加位置参数：输入文件路径
    parser.add_argument("input_file", help="Path to the input JSON file")
    
    # 添加位置参数：输出文件路径
    parser.add_argument("output_file", help="Path to the output JSON file")
    
    # 添加可选参数：要处理的记录数量，默认为1000
    parser.add_argument("--num_entries", type=int, default=1000, help="Number of entries to process")

    # 解析命令行参数
    args = parser.parse_args()
    
    # 调用convert_json_format函数进行实际的数据转换
    convert_json_format(args.input_file, args.output_file, args.num_entries)
```

执行命令对文件进行转换与生成

python convert_json.py ./data/medicalQA.json ./data/medicalQA_top1k.json --num_entries 1000 

head -n 10 ./data/medicalQA_top1k   执行完成后进行数据校验

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224211610270.png" width=100%></div>

- **step2 开始模型微调**  

依赖版本信息：

| 库             | 版本         | 
| -------------- | ------------ | 
| Python         | 3.12.2       | 
| pytorch          | 2.4.0+cu121  | 
| datasets       | 3.0.0        | 
| pandas         | 2.2.2        | 
| transformers   | 4.44.0       | 
| peft           | 0.12.0       |
|tensorboard |2.18.0|

大部分适用pip install 命令安装即可 

查看安装包版本命令：

pip show 安装包名称

python -c "import torch; print(torch.__version__)"

### 1 多GPU微调：

本次微调方式为LORA，可以根据个人需求精简输入参数

代码命名为： train_distributed.py
```python 
# 导入所需的库和模块
import os  # 导入操作系统接口模块 它可以用来访问环境变量、处理文件路径、操作文件系统等。
import json  # 导入JSON处理模块 能够将 Python 对象转换为 JSON 字符串，或将 JSON 字符串转换为 Python 对象，常用于数据交换
import torch  # 导入PyTorch库，用于张量操作和深度学习  是 PyTorch 的核心库
import torch.distributed as dist  # 导入PyTorch分布式训练支持  是 PyTorch 用于分布式训练的模块。它提供了多种方式来在多台机器或多张 GPU 之间同步和共享数据，用于加速模型训练
from datasets import Dataset  # Hugging Face 提供的一个库，用于加载和处理数据集。它提供了许多常用的自然语言处理（NLP）和机器学习任务的数据 
import pandas as pd  # Python 中最常用的数据处理库，特别擅长处理结构化数据（例如 CSV 文件或数据库表格）。它提供了高效的 DataFrame 结构，适合用于数据清洗、数据分析等任务。
from transformers import (  # 是 Hugging Face 提供的一个库，用于加载和训练预训练的自然语言处理（NLP）模型
    AutoModelForCausalLM,  # 自动加载因果语言模型（例如 GPT、GPT-2、GPT-3）。
    AutoTokenizer,  # 自动加载分词器，它将文本转换为模型能够理解的 token（标记）表示。
    TrainingArguments,  # 用于配置训练过程的各种参数（如批次大小、学习率等）。
    Trainer,  # 用于训练模型的高级接口，简化了模型训练流程。
    DataCollatorForSeq2Seq  #用于处理序列到序列任务（如翻译、摘要等）的数据整理器。
)
 # 用于参数高效微调（PEFT，Parameter Efficient Fine-Tuning）的库，支持 LoRA等方法。
from peft import (get_peft_model, # 用于获取支持 LoRA 的模型
                  LoraConfig, # 配置 LoRA 模型的超参数（如 rank、alpha 和 dropout 等）
                  TaskType, # 指定任务类型（例如因果语言模型任务）。
                  prepare_model_for_kbit_training )  #用于将模型准备为适合进行低比特训练（如 8bit）的格式
from argparse import ArgumentParser  # 用于解析命令行参数。在这里，它用于解析命令行中输入的各种参数，如模型路径、数据路径、训练超参数等。
from torch.utils.tensorboard import SummaryWriter #SummaryWriter 用于将训练过程中的日志信息（如损失值、学习率、准确率等）写入 TensorBoard 可视化工具中。
import datetime #提供了用于操作日期和时间的类

# 定义一个自定义的Trainer类，继承自Hugging Face的Trainer
class CustomTrainer(Trainer):
     # 初始化方法，初始化Trainer的基础功能并扩展
    def __init__(self, *args,tensorboard_dir=None , **kwargs):
        # 调用父类Trainer的构造函数，初始化基本功能
        super().__init__(*args, **kwargs)
        # 启用梯度检查点，以减少内存使用
        # 这将使得在训练过程中某些中间激活值不被存储，而是根据需要重新计算，从而节省内存
        self.model.gradient_checkpointing_enable()
        
        # 创建 TensorBoard writer，用于记录训练过程中的各类指标
        # 通过当前时间戳生成唯一的日志目录
        current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')  # 获取当前时间并格式化
        self.writer = SummaryWriter(f'{tensorboard_dir}/training_{current_time}') 
        
    # 自定义的损失计算方法
    # 这个方法会覆盖Trainer中的compute_loss方法，用于记录损失及其他指标
    def compute_loss(self, model, inputs, return_outputs=False):
        # 使用模型计算输出，传入inputs是模型的输入数据
        outputs = model(**inputs)
        
        # 获取损失值，这通常是模型的输出中的 `loss` 字段
        loss = outputs.loss
        
        # 每当global_step达到记录间隔时，记录信息到TensorBoard
        if self.state.global_step % self.args.logging_steps == 0:
            try:
                # 记录训练损失到TensorBoard，只有在损失值是有效的情况下
                if not torch.isnan(loss) and not torch.isinf(loss):
                    self.writer.add_scalar('Training/Loss', 
                                         loss.item(),  # 损失的值
                                         self.state.global_step)  # 当前的训练步数
                
                # 记录当前学习率到TensorBoard
                self.writer.add_scalar('Training/Learning_Rate',
                                     self.optimizer.param_groups[0]['lr'],  # 当前学习率
                                     self.state.global_step)  # 当前的训练步数
                
                # 记录训练进度（百分比），计算进度=当前步数/最大步数*100
                self.writer.add_scalar('Training/Progress',
                                     self.state.global_step / self.args.max_steps * 100,  # 计算训练进度
                                     self.state.global_step)  # 当前的训练步数
                
                # 记录批量大小（即每次训练使用的样本数）
                self.writer.add_scalar('Training/Batch_Size', 
                                     inputs['input_ids'].size(0),  # 输入的批量大小
                                     self.state.global_step)  # 当前的训练步数
                
                # 如果支持GPU，则记录GPU的内存使用情况
                if torch.cuda.is_available():
                    # 记录当前GPU分配的内存（单位MB）
                    self.writer.add_scalar('System/GPU_Memory_Allocated',
                                         torch.cuda.memory_allocated() / 1024**2,  # GPU内存分配量
                                         self.state.global_step)
                    # 记录当前GPU保留的内存（单位MB）
                    self.writer.add_scalar('System/GPU_Memory_Reserved',
                                         torch.cuda.memory_reserved() / 1024**2,  # GPU内存保留量
                                         self.state.global_step)
                
                # 安全地记录梯度直方图，每隔一定步数记录一次梯度信息
                if self.state.global_step % (self.args.logging_steps * 10) == 0:  # 降低记录频率
                    # 遍历模型中所有的可训练参数
                    for name, param in model.named_parameters():
                        if param.requires_grad and param.grad is not None:
                            grad_data = param.grad.data
                            # 检查梯度数据是否有效（即不为空或全为零）
                            if grad_data.numel() > 0 and not torch.all(grad_data == 0):
                                try:
                                    # 将梯度的直方图记录到TensorBoard
                                    self.writer.add_histogram(
                                        f'Gradients/{name}',  # 记录的名称，包括参数的名称
                                        grad_data.float().cpu().numpy(),  # 转换为CPU上的float类型并转为numpy数组
                                        self.state.global_step,  # 当前的训练步数
                                        bins='auto'  # 自动确定直方图的bins数量
                                    )
                                except Exception as e:
                                    print(f"Warning: Failed to log histogram for {name}: {e}")
                
                # 计算模型所有参数的梯度范数并记录到TensorBoard
                total_norm = 0.0  # 初始化总范数
                for p in model.parameters():
                    if p.grad is not None:
                        param_norm = p.grad.data.norm(2)  # 计算当前参数的L2范数
                        total_norm += param_norm.item() ** 2  # 累加所有参数的L2范数的平方
                total_norm = total_norm ** 0.5  # 最终的梯度范数是所有参数L2范数平方和的平方根
                self.writer.add_scalar('Gradients/total_norm',  # 记录总的梯度范数
                                     total_norm,
                                     self.state.global_step)  # 当前的训练步数
                    
            except Exception as e:
                print(f"Warning: Error in logging to tensorboard: {e}")
        
        # 处理 NaN/Inf 损失，确保损失值有效
        if torch.isnan(loss).any() or torch.isinf(loss).any():
            print(f"Warning: NaN/Inf loss detected: {loss.item()}")
            # 将NaN/Inf损失值替换为0
            loss = torch.where(torch.isnan(loss), torch.full_like(loss, 0.0), loss)
            loss = torch.where(torch.isinf(loss), torch.full_like(loss, 0.0), loss)
    
        # 返回计算出的损失，如果需要返回输出，则返回损失和模型输出
        return (loss, outputs) if return_outputs else loss



    def get_grad_norm(self):
        """
        计算模型中所有参数的梯度范数 
        梯度范数是衡量梯度大小的一种方法，有助于检测梯度爆炸或梯度消失问题。
        """
        total_norm = 0.0  # 初始化总的梯度范数为0
        # 遍历模型中的所有参数
        for p in self.model.parameters():
            # 只计算需要梯度的参数，并且检查这些参数的梯度是否为None
            if p.requires_grad and p.grad is not None:
                # 计算当前参数梯度的2范数（即L2范数）
                param_norm = p.grad.data.norm(2)
                # 将该参数的梯度范数平方加到总范数中
                total_norm += param_norm.item() ** 2
        # 计算所有参数梯度的总范数的平方根，即L2范数
        total_norm = total_norm ** (1. / 2)
        return total_norm  # 返回计算得到的梯度范数

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """
        评估回调函数，记录验证集的评估指标
        在每次评估结束时，会将评估指标记录到 TensorBoard 中，以便后续分析。
        """
        if metrics:
            # 如果metrics不为空，遍历所有指标
            for key, value in metrics.items():
                # 记录每个评估指标到TensorBoard，其中 'Eval' 是评估阶段的前缀
                self.writer.add_scalar(f'Eval/{key}', value, state.global_step)


    def on_train_end(self, args, state, control, **kwargs):
        """
        在训练结束时，我们应该关闭 `SummaryWriter`，以确保所有数据都已经写入并保存。
        """
        self.writer.close()  # 关闭TensorBoard写入器，释放资源

# 设置分布式训练环境的函数
def setup_distributed():
    local_rank = int(os.environ.get("LOCAL_RANK", -1))
    world_size = int(os.environ.get("WORLD_SIZE", -1))  # 获取分布式训练中的世界大小（总的训练进程数）
    
    if local_rank != -1: 
        torch.cuda.set_device(local_rank)  # 设置当前进程使用的GPU设备
        dist.init_process_group(backend="nccl")  # 初始化进程组，使用NCCL作为通信后端

    return local_rank, world_size


# 数据预处理函数，将输入文本转换为模型所需的格式
    # 该函数从 instruction、input 和 output 字段构建一个完整的提示语（prompt）。如果 input 存在，它包含 input，否则只包含 instruction 和 output。
    # 使用tokenizer 对提示语进行编码，转换为模型可以处理的 input_ids（token ids），并进行填充或截断。
    # 构建labels（即标签），将pad_token_id替换为-100，表示这些部分不参与损失计算。
def preprocess_function(examples, tokenizer, max_length=512):    
    prompts = []  # 存储构建好的所有prompt（模型输入格式）
    
    # 将每一条数据转换为模型输入格式
    for instruction, input_text, output in zip(examples['instruction'], examples['input'], examples['output']):
        # 如果输入文本存在且不为空，构建带输入的完整提示语
        if input_text and input_text.strip():
            prompt = f"Input: {input_text}\nInstruction: {instruction}\nOutput: {output}"  # 输入存在时，构建完整的提示语
        else:
            # 如果输入为空，则只包含指令和输出
            prompt = f"Instruction: {instruction}\nOutput: {output}" 
        
        prompts.append(prompt)  # 将构建好的prompt添加到prompts列表

    # 使用tokenizer将prompt列表转化为模型输入的token ids，并进行填充、截断
    tokenized_inputs = tokenizer(
        prompts,  # 将所有的prompts转化为tokens
        max_length=max_length,  # 最大长度，超过部分会被截断
        truncation=True,  # 如果文本超过max_length，进行截断
        padding='max_length',  # 填充到最大长度
        return_tensors="pt"  # 返回PyTorch张量
    )

    # 将tokenized_inputs的input_ids用作标签，并将pad_token_id替换为-100，表示这些部分不参与损失计算
    labels = tokenized_inputs['input_ids'].clone()  # 克隆input_ids，作为标签
    labels[labels == tokenizer.pad_token_id] = -100  # 将pad_token_id替换为-100，以防计算损失时被考虑
    tokenized_inputs['labels'] = labels  # 将标签添加到tokenized_inputs字典中
    
    return tokenized_inputs  # 返回处理后的tokenized输入（包括input_ids, attention_mask, labels）


# 将JSON格式数据转换为Dataset格式并进行预处理
    # 数据加载：通过读取JSON文件，将其转换为Python字典。
    # 数据清洗：过滤掉没有有效 instruction 或 output 的样本，确保这些字段为有效文本。
    # 转换为Dataset：通过 Dataset.from_pandas 将清洗后的数据转换为Hugging Face的 Dataset 格式。
    # 数据预处理：使用 map 函数将 preprocess_function 应用于每个数据项，生成 input_ids、attention_mask 和 labels。
    # 保存数据集：将处理好的数据集保存到指定目录，以便后续使用
def convert_json_to_dataset(json_file_path, save_directory, tokenizer):
    
    # 打开JSON文件并加载数据
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)  # 加载JSON文件内容
    
    # 数据清洗：确保每一项的数据中'input'和'output'字段不为空
    cleaned_data = []
    for item in data:
        # 检查instruction和output是否为空（也去除空格）
        if all(isinstance(v, str) and len(v.strip()) > 0 for v in [item['instruction'], item['output']]):
            cleaned_data.append(item)  # 如果有效，则添加到cleaned_data中
    
    # 将清洗后的数据转换为Dataset格式
    dataset = Dataset.from_pandas(pd.DataFrame(cleaned_data))  # 使用pandas DataFrame格式转换为Dataset对象
    
    # 使用map函数对数据进行预处理，每一条数据会通过preprocess_function进行处理
    processed_dataset = dataset.map(
        lambda x: preprocess_function(x, tokenizer),  # 调用预处理函数
        batched=True,  # 一次处理多个样本
        remove_columns=dataset.column_names  # 去掉原始的列，保留处理后的数据
    )
    
    # 设置数据格式为torch，保留input_ids, attention_mask和labels
    processed_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    # 保存处理后的数据集到指定目录
    if not os.path.exists(save_directory):  # 如果保存目录不存在
        os.makedirs(save_directory)  # 创建目录
    processed_dataset.save_to_disk(save_directory)  # 将数据集保存到磁盘
    
    return processed_dataset  # 返回处理好的数据集


# 准备模型和分词器的函数 做了以下事情：
# 加载指定路径或名称的分词器。
# 配置模型加载到正确的设备上（支持单卡和分布式训练）。
# 使用低精度（如 8bit 和 float16）加载模型，以减少内存消耗。
# 调用 prepare_model_for_kbit_training 函数，为低位训练做好准备。
# 返回加载好的模型和分词器。
def prepare_model_and_tokenizer(model_path, local_rank):
    
    # 使用AutoTokenizer从预训练模型加载分词器
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,  # 指定模型路径或模型名称
        trust_remote_code=True  # 允许从远程加载自定义代码（如自定义分词器）
    )
    
    # 设置模型设备映射，确保模型加载到正确的设备
    if local_rank == -1:
        # 如果不进行分布式训练，则将模型加载到当前GPU设备
        device_map = {'': torch.cuda.current_device()}  # 默认设备（当前GPU）
    else:
        # 如果进行分布式训练，则根据local_rank指定加载到特定GPU
        device_map = {'': local_rank}  # 在分布式训练中指定设备ID
    
    # 加载预训练的Causal LM模型
    model = AutoModelForCausalLM.from_pretrained(
        model_path,  # 模型路径或名称
        trust_remote_code=True,  # 允许从远程加载自定义代码（如自定义模型配置）
        load_in_8bit=True,  # 使用8位精度进行加载，减少内存消耗
        torch_dtype=torch.float16,  # 使用float16精度，进一步节省显存
        device_map=device_map  # 使用指定的设备映射
    )
    
    # 准备模型进行低位训练（k-bit training）
    # 低位训练通常用于减少模型的存储需求，并提高推理速度。
    # 低位训练：在训练中使用低精度的权重（如 4bit 或 8bit）来减少内存占用
    model = prepare_model_for_kbit_training(model) 
    
    # 返回加载好的模型和分词器
    return model, tokenizer  

# 主函数，执行训练流程
    """
    1.解析命令行参数：
        通过 ArgumentParser 解析用户提供的命令行参数。这些参数包括模型路径、数据路径、训练超参数、分布式训练配置等。
    2.设置分布式训练：
        根据 local_rank 设置分布式训练环境。如果启用了分布式训练，则会在多个设备（GPU）之间分配任务。is_main_process 用于判断当前进程是否为主进程，主进程通常负责保存模型。
    3.准备模型和分词器：
        使用 prepare_model_and_tokenizer 函数加载预训练模型和分词器，并根据 local_rank 设置合适的设备映射。
    4.配置训练参数：
        设置训练参数，如学习率、批次大小、梯度累积步数、最大训练步数等，此外还配置了日志记录、保存模型等选项。
    5.配置LoRA：
        配置并应用LoRA（Low-Rank Adaptation）到模型中，以便进行低秩微调。
    6.加载或创建数据集：
        检查数据集是否已存在（保存在 save_directory 中），如果存在则加载；如果不存在，则从提供的 JSON 文件中生成数据集。
    7.训练：
        使用自定义的 CustomTrainer 类来执行模型训练。这个类集成了常见的训练任务，例如损失计算、日志记录、梯度计算等。
    8.保存模型：
        训练完成后，主进程会保存训练好的模型。
    9.清理分布式环境：
        在分布式训练结束后，清理分布式进程组。
    """
def main():
    # 解析命令行参数，获取用户输入的超参数和路径配置
    parser = ArgumentParser()
    
    # 模型路径和数据路径参数
    parser.add_argument("--model_path", type=str, default=None, help="模型路径")
    parser.add_argument("--json_file_path", type=str, default=None, help="数据集路径")
    parser.add_argument("--save_directory", type=str, default="./data/train_data_distribute", help="数据保存路径")
    parser.add_argument("--output_dir", type=str, default="./output", help="训练输出路径")
    parser.add_argument("--log_path", type=str, default="./logs", help="日志输出路径")
    
    # 分布式训练参数
    parser.add_argument("--local_rank", type=int, default=-1,help="可以自行选择是否传参，也可以外部CUDA控制")
    
    # 训练超参数
    parser.add_argument("--batch_size", type=int, default=2, help="每个设备的训练批次大小")
    parser.add_argument("--gradient_accumulation_steps", type=int, default=8, help="梯度累积步数")
    parser.add_argument("--learning_rate", type=float, default=5e-6, help="学习率")
    parser.add_argument("--max_steps", type=int, default=50000, help="训练的最大步数建议设置大一点")
    parser.add_argument("--warmup_steps", type=int, default=100, help="预热步骤数")
    parser.add_argument("--logging_steps", type=int, default=10, help="记录日志的步骤数")
    parser.add_argument("--save_steps", type=int, default=100, help="保存模型的步骤数")
    parser.add_argument("--train_epochs", type=int, default=10, help="训练轮数")
    parser.add_argument("--max_grad_norm", type=float, default=0.5, help="最大梯度裁剪值")
    parser.add_argument("--fp16", type=bool, default=True, help="选择位数")
    
    # LoRA配置
    parser.add_argument("--rank", type=int, default=8, help="LoRA配置中的r值")
    parser.add_argument("--lora_alpha", type=int, default=32, help="LoRA配置中的alpha值")
    parser.add_argument("--lora_dropout", type=float, default=0.05, help="LoRA配置中的dropout值")
    parser.add_argument("--tensorboard_dir", type=str, default="tensorboard", help="TensorBoard 日志目录")
    
    # 解析命令行参数
    args = parser.parse_args()

    # 确保 TensorBoard 日志目录存在
    os.makedirs(args.tensorboard_dir, exist_ok=True)  # 创建目录（如果不存在）
    
    # 设置分布式训练环境（如果启用）
    local_rank, world_size = setup_distributed()  # 设置分布式训练的环境变量
    is_main_process = local_rank in [-1, 0]  # 判断当前进程是否为主进程

    # 准备模型和分词器
    model, tokenizer = prepare_model_and_tokenizer(args.model_path, local_rank)  # 加载预训练模型和分词器

    # 配置训练参数
    training_args = TrainingArguments(
        output_dir=args.output_dir,
        per_device_train_batch_size=args.batch_size,  # 每个GPU的训练批次大小
        gradient_accumulation_steps=args.gradient_accumulation_steps,  # 梯度累积步数
        learning_rate=args.learning_rate,  # 学习率
        fp16=args.fp16,  # 是否使用16位精度训练
        logging_steps=args.logging_steps,  # 日志记录的步数间隔
        save_steps=args.save_steps,  # 模型保存的步数间隔
        max_steps=args.max_steps,  # 最大训练步数
        warmup_steps=args.warmup_steps,  # 预热步数
        logging_dir=args.log_path,  # TensorBoard日志路径
        num_train_epochs=args.train_epochs,  # 训练轮数
        save_strategy="epoch",  # 每个epoch保存一次模型
        save_total_limit=3,  # 最大保存模型的数量
        remove_unused_columns=False,  # 保留所有列
        local_rank=local_rank,  # 分布式训练中使用的local rank
        ddp_backend="nccl",  # 分布式训练时使用的后端
        dataloader_num_workers=2,  # 数据加载的工作线程数
        gradient_checkpointing=True,  # 启用梯度检查点，减少显存消耗
        max_grad_norm=args.max_grad_norm,  # 梯度裁剪的最大值
        ddp_find_unused_parameters=False,  # 在分布式训练时，不查找未使用的参数
        report_to="tensorboard"  # 启用TensorBoard报告
    )

    # 配置LoRA（Low-Rank Adaptation）设置
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,  # 任务类型：因果语言模型
        r=args.rank,  # LoRA的rank值
        lora_alpha=args.lora_alpha,  # LoRA的alpha值
        lora_dropout=args.lora_dropout,  # LoRA的dropout值
        target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],  # 适用于这些模块的LoRA
        bias="none",  # 不使用偏置
        inference_mode=False  # 非推理模式，表示用于训练
    )

    # 将LoRA配置应用到模型中
    model = get_peft_model(model, lora_config)  # 获取带有LoRA配置的模型

    # 加载数据集，如果已存在则直接加载，否则从JSON文件创建
    if os.path.exists(args.save_directory):
        dataset = Dataset.load_from_disk(args.save_directory)  # 加载已保存的数据集
    else:
        dataset = convert_json_to_dataset(args.json_file_path, args.save_directory, tokenizer)  # 从JSON文件转换为Dataset格式

    # 使用自定义Trainer进行训练
    trainer = CustomTrainer(
        model=model,  # 传入模型
        args=training_args,  # 传入训练参数
        train_dataset=dataset,  # 传入训练数据集
        tensorboard_dir=args.tensorboard_dir,  # TensorBoard日志目录
        data_collator=DataCollatorForSeq2Seq(
            tokenizer,  # 使用的tokenizer
            pad_to_multiple_of=8,  # 填充到8的倍数
            return_tensors="pt",  # 返回PyTorch张量
            padding=True  # 启用填充
        )
    )

    # 开始训练
    trainer.train()

    # 只在主进程保存模型
    if is_main_process:
        trainer.save_model(f"{args.output_dir}/final_model")  # 保存最终的训练模型

    # 清理分布式环境
    if local_rank != -1:
        dist.destroy_process_group()  # 销毁分布式训练的进程组

# 程序入口，调用主函数
if __name__ == "__main__":
    main()  # 调用主函数，启动训练流程

```

执行命令参数：

脚本命名为train_distributed.sh 
```bash
#!/bin/bash
# 指定使用 bash 作为脚本的解释器

# 获取当前系统中可用的 GPU 数量
NUM_GPUS=$(nvidia-smi --query-gpu=gpu_name --format=csv,noheader | wc -l)
# 解释：通过 `nvidia-smi` 命令查询所有 GPU 的名称，然后使用 `wc -l` 统计 GPU 的数量
# 这个数量会赋值给变量 NUM_GPUS，用于后续指定训练时使用的 GPU 数量

# 如果需要，可以使用以下命令指定要使用的显卡
# export CUDA_VISIBLE_DEVICES="0,1" 
# 这个命令允许指定哪些 GPU 被用来训练（例如，使用 GPU 0 和 GPU 1）

# 使用 `torchrun` 启动分布式训练
# `torchrun` 是 PyTorch 用于分布式训练的工具，可以自动管理多卡训练

  # --nproc_per_node=$NUM_GPUS：指定每个节点使用的 GPU 数量，$NUM_GPUS 是之前获取的 GPU 数量
  # --nnodes=1：表示使用一个节点进行训练
  # --node_rank=0：指定当前节点的 rank（编号），主节点通常是 0
  # 指定训练时使用的各个参数
torchrun --nproc_per_node=$NUM_GPUS  --nnodes=1  --node_rank=0 train_distributed.py \
--model_path "/home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct" \
--json_file_path "/home/data/muyan/code/data/medicalQA_top1k.json" \
--save_directory "./data/train_data_distribute_20241224" \
--output_dir "./output_20241224" \
--log_path "./log_20241224" \
--tensorboard_dir "tensorboard_dir" \
--batch_size 2 \
--train_epochs 10 \
--fp16 True \
--gradient_accumulation_steps 8 \
--learning_rate 5e-5 \
--max_steps -1 \
--warmup_steps 0 \
--logging_steps 10 \
--save_steps 100 \
--max_grad_norm 0.5 \
--rank 8 \
--lora_alpha 16 \
--lora_dropout 0.05

# 以上命令在使用时注意格式 比如 `\`后不能有空格，可能会导致命令无法传入程序
# 当前参数在使用时需要根据实际数据集效果自行进行调整
# torchrun --nproc_per_node=$NUM_GPUS  --nnodes=1  --node_rank=0 train_distributed.py \
# --model_path "/home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct" \  # 预训练模型路径
# --json_file_path "/home/data/muyan/code/data/medicalQA_top1k.json" \  # 训练数据集路径
# --save_directory "./data/train_data_distribute_20241224" \  # 数据保存目录
# --output_dir "./output_20241224" \  # 模型输出结果保存目录
# --log_path "./log_20241224" \  # 日志文件保存路径
# --tensorboard_dir "tensorboard_dir" \ # 指定为当前路径下 tensorboard_dir文件夹
# --batch_size 2 \  # 每个 GPU 上的训练批次大小
# --train_epochs 100 \  # 训练的轮数
# --fp16 True \  # 启用 16 位精度训练（减少显存占用并加速训练）
# --gradient_accumulation_steps 8 \  # 梯度累积步数（每 8 步进行一次梯度更新）
# --learning_rate 5e-5 \  # 学习率
# --max_steps -1 \  # 最大训练步数，设置为 -1 表示使用 `train_epochs` 控制训练轮数
# --warmup_steps 0 \  # 学习率预热的步数，设置为 0 表示不使用预热
# --logging_steps 10 \  # 每 10 步记录一次训练日志
# --save_steps 100 \  # 每 100 步保存一次模型
# --max_grad_norm 0.5 \  # 最大梯度裁剪值，用于防止梯度爆炸
# --rank 8 \  # LoRA 配置中的 r 值，控制低秩矩阵的大小
# --lora_alpha 16 \  # LoRA 配置中的 alpha 值，控制低秩矩阵的缩放
# --lora_dropout 0.05  # LoRA 配置中的 dropout 值，用于防止过拟合
```

执行命令：

./train_distributed.sh

如果权限不足请执行：chmod +x train_distributed.sh

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224221355643.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224221853942.png" width=100%></div>

### 过程监控：

生成传入文件指定路径

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224222054302.png" width=100%></div>

tensorboard安装及使用在《Ch 2. LLama_Factory+LORA大模型微调》已经讲过，本次不在赘述

监控命令(指定文件夹与当前IP地址和开放端口号)：tensorboard --logdir=tensorboard_dir --host 192.168.110.131  --port=6006


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224222428722.png" width=100%></div>

界面查看

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224222551642.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224222840548.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224222943296.png" width=100%></div>

后台监控数据 

强烈建议大家使用 nohup 命令启动文件

nohup ./train_distributed.sh >train_distributed_12242315.log 2>&1 &

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224231601852.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224231700295.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224223156093.png" width=100%></div>

进入指定路径查看生成文件

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224223650177.png" width=100%></div>

生成数据的基本信息

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224223834722.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224223941145.png" width=100%></div>

| Data format | Function                |
|-------------|-------------------------|
| Arrow       | Dataset.save_to_disk()  |
| CSV         | Dataset.to_csv()        |
| JSON        | Dataset.to_json()       |

Apache Arrow是一种用于高效存储和交换表格数据的列式存储格式

Arrow格式专为高性能设计，支持零拷贝读取和写入，这可以显著提高数据的处理速度

Arrow格式使用列式存储，可以更有效地使用内存，特别是对于大型数据集

生成日志查看

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224224355743.png" width=100%></div>

模型保存位置：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224224600642.png" width=100%></div>

指定tensorboard 文件路径

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241224224921036.png" width=100%></div>

显存占用监控（当前是指定使用两张卡进行微调0,1 不指定某一张卡则会使用全部的卡进行微调）

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241223172302664.png" width=100%></div>

- **step3 检验模型输出**

### 加载方式验证

命名为lora_adapter_chat.py
```python
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import warnings

# 关闭所有警告，避免干扰输出
warnings.filterwarnings('ignore')

class ChatBot:
    def __init__(self, base_model_path, lora_model_path):
        """
        初始化聊天机器人，加载基础模型和LoRA模型。
        
        参数:
        base_model_path (str): 基础模型路径，通常是预训练的语言模型路径。
        lora_model_path (str): LoRA模型路径，用于微调。
        """
        print("正在加载模型...")
        
        # 加载tokenizer：用于将输入文本转换为模型可以理解的tokens（标记），并将模型的输出转换为可读的文本。
        self.tokenizer = AutoTokenizer.from_pretrained(
            base_model_path,  # 基础模型路径
            trust_remote_code=True  # 允许加载远程的代码
        )
        
        # 加载基础模型：加载预训练的因果语言模型，用于生成文本。
        self.base_model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            trust_remote_code=True,  # 允许加载远程的代码
            device_map="auto",  # 自动选择设备（如GPU/CPU）
            torch_dtype=torch.float16  # 使用16位精度来减少内存占用
        )
        
        # 加载LoRA模型：基于LoRA微调的模型，用于增强生成能力。
        self.model = PeftModel.from_pretrained(
            self.base_model,  # 基础模型
            lora_model_path,  # LoRA微调后的模型路径
            torch_dtype=torch.float16,  # 使用16位精度
            device_map="auto"  # 自动选择设备
        )
        
        print("模型加载完成！")
        
    def generate_response(self, instruction, input_text=""):
        """
        根据给定的指令和输入文本生成回复。
        
        参数:
        instruction (str): 聊天机器人执行的指令。
        input_text (str, optional): 用户输入的文本，默认为空。

        返回:
        response (str): 生成的回复。
        """
        # 根据是否有输入文本构建不同的提示语（prompt）
        if input_text:
            prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput:"
        else:
            prompt = f"Instruction: {instruction}\nOutput:"
        
        # 使用tokenizer将prompt转换为模型输入格式
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}  # 将输入移动到与模型相同的设备
        
        # 禁用梯度计算（推理时不需要计算梯度）
        with torch.no_grad():
            # 生成模型的输出（回复）
            outputs = self.model.generate(
                **inputs,  # 输入的token ids
                max_new_tokens=512,  # 最多生成512个新tokens
                temperature=0.1,  # 控制生成的多样性，较低的温度使输出更确定性
                top_p=0.1,  # 控制样本的概率分布
                repetition_penalty=1.1,  # 重复惩罚，避免生成重复文本
                pad_token_id=self.tokenizer.pad_token_id,  # 填充token的ID
                eos_token_id=self.tokenizer.eos_token_id  # 结束token的ID
            )
        
        # 解码生成的tokens为文本，并去除特殊tokens
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 提取输出部分的回复文本，去除"Output:"标记
        response = response.split("Output:")[-1].strip()
        
        return response

def main():
    """
    主函数：用于启动聊天机器人，接收用户输入，并生成回复。
    """
    # 配置基础模型路径和LoRA模型路径
    BASE_MODEL_PATH = "/home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct"  # 替换为你的基础模型路径
    LORA_MODEL_PATH = "./output_20241224/final_model"  # 替换为你的LoRA模型路径
    
    # 初始化聊天机器人
    chatbot = ChatBot(BASE_MODEL_PATH, LORA_MODEL_PATH)
    
    # 欢迎信息和说明
    print("\n=== 欢迎使用聊天机器人 ===")
    print("输入 'quit' 或 'exit' 结束对话")
    print("输入 'clear' 清空对话历史")
    print("============================\n")
    
    while True:
        try:
            # 获取用户输入的指令
            instruction = input("\n请输入指令: ").strip()
            
            # 检查是否退出对话
            if instruction.lower() in ['quit', 'exit']:
                print("\n感谢使用！再见！")
                break
                
            # 检查是否清空对话历史
            if instruction.lower() == 'clear':
                print("\n对话历史已清空！")
                continue
                
            # 获取用户输入的可选文本
            input_text = input("请输入补充内容(可选，直接回车跳过): ").strip()
            
            # 生成机器人回复
            print("\n正在生成回复...")
            response = chatbot.generate_response(instruction, input_text)
            
            # 输出机器人的回复
            print("\n机器人回复:")
            print("-" * 50)
            print(response)
            print("-" * 50)
            
        except KeyboardInterrupt:
            print("\n\n检测到键盘中断，正在退出...")
            break
            
        except Exception as e:
            print(f"\n发生错误: {str(e)}")
            print("请尝试重新输入或输入 'exit' 退出")
            continue

# 启动程序入口
if __name__ == "__main__":
    main()

```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225104801873.png" width=100%></div>

- **step4 合并模型输出**

```python
# 当前文件 命名为merge_model.py
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. 加载原始基座模型（Qwen2.5-3B-Instruct）
base_model_path = "/home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_path)

# 2. 加载微调后的 LoRA adapter（假设你使用的是 LoRA 库）
adapter_path = "output_20241224/23/final_model/"
# 这里需要根据你使用的 LoRA 实现来加载 adapter
# 例如，如果你使用的是 LoRA 模块并且是 Hugging Face 实现：
from peft import PeftModel  # 这里假设你使用的是 `peft` 库，具体取决于你的 LoRA 实现

# 加载微调后的 LoRA adapter
adapter = PeftModel.from_pretrained(model, adapter_path)

# 3. 合并 LoRA adapter 和基座模型
#使用的是 LoRA 库，通常调用 `PeftModel` 的方法会自动将 adapter 应用到基座模型上

# 4. 保存合并后的模型
final_model_path = "merge_model/1224/24_23_0.5B_200"
adapter.save_pretrained(final_model_path)

# 你也可以保存 tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"模型已保存至 {final_model_path}")
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225152233200.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225194811534.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225152742792.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225153030755.png" width=100%></div>

- **step5 模型效果校验**

当前使用qwen2.5本地执行web界面验证

当前模型为0.5B 200 epoch合并后模型

现在你是一名专业的中医医生，请用你的专业知识提供详尽而清晰的关于中医问题的回答。这段时间去上厕所本来想小便的可是每次都会拉大便。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225115351511.png" width=100%></div>

现在你是一名专业的中医医生，请用你的专业知识提供详尽而清晰的关于中医问题的回答。你好，全身没劲，没精神，吃不下饭，只想睡觉，是什么情况。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225115745203.png" width=100%></div>

原生模型

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225120234766.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225120329200.png" width=100%></div>

# 5. 扩展使用

## 5.1 deepspeed微调

增加两行代码 一个文件

parser.add_argument("--deepspeed", type=str, default="ds_config.json", help="DeepSpeed 配置文件路径")

deepspeed=args.deepspeed,

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225204213883.png" width=100%></div>

```json
{
    "fp16": {
        "enabled": true,
        "auto_cast": true
    },
    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": true,
        "overlap_comm": true,
        "reduce_scatter": true,
        "contiguous_gradients": true
    },
    "gradient_accumulation_steps": "auto",
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_clipping": "auto"
}
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225205404809.png" width=100%></div>

感觉麻烦的同学可以去 LLaMA-Factory/cache/中复制

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225205636375.png" width=100%></div>

代码重新命名脚及代码后边都增加 _deepspeed 例如 ：train_distributed_deepspeed.py

启动命令和上边一样 ./train_distributed_deepspeed.py


可以使用 也可以使用 torchrun
```bash
deepspeed --num_gpus=4 train_distributed_deepspeed.py \
    --model_path "/home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct" \
    --json_file_path "/home/data/muyan/code/data/medicalQA_top1k.json" \
    --output_dir "./output" \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 8 \
    --learning_rate 5e-6 \
    --max_steps 500 
```

`DeepSpeed` 是一个专门为大规模分布式深度学习任务设计的训练加速框架。相比 `torchrun`，它不仅支持分布式训练，还提供了许多高级特性，如 ZeRO 优化器、梯度压缩等。

- **支持更多优化功能**：
  - 混合精度训练（FP16/FP8）。
  - ZeRO 优化（分布式显存优化）。
  - 动态张量切分。
  - CPU 和 NVMe 混合内存支持。


| 启动方式        | 分布式支持         | 适用场景                      | 优势                          | 限制                          |
|-----------------|-------------------|-------------------------------|-------------------------------|-------------------------------|
| **python**      | 无（默认单进程）   | 单 GPU 或简单任务              | 简单易用，快速调试代码          | 不支持多 GPU 或多节点          |
| **torchrun**    | 内置分布式支持     | 单节点多 GPU 或多节点分布式任务  | 易用、支持 DDP 等分布式功能     | 不支持大模型优化和显存节省      |
| **deepspeed**   | 高级分布式支持     | 大规模分布式或大模型训练        | 显存优化（ZeRO）、大模型支持     | 依赖配置文件，学习成本较高      |



对于类似更多deepspeed 参数传递 可以参考源码注释进行配置：https://github.com/huggingface/transformers/blob/24c91f095fec4d90fa6901ef17146b4f4c21d0a3/src/transformers/training_args.py#L223 

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241225212502598.png" width=100%></div>

## 5.2 多轮数据处理

```python 
# 当前代码仅用做思路参考，具体需要根据使用状况进行调整使用，数据集样例为：
    # {
    #     "instruction": "请根据聊天历史和当前问题回答",
    #     "input": "今天天气怎么样？",
    #     "output": "根据之前的对话内容，今天是晴天，气温适宜。",
    #     "history": [
    #         {
    #             "human": "你好，能告诉我今天的天气预报吗？",
    #             "assistant": "今天是晴天，气温在20-25度之间，很适合外出活动。"
    #         },
    #         {
    #             "human": "需要带伞吗？",
    #             "assistant": "不需要带伞，今天不会下雨，天气很好。"
    #         }
    #     ]
    # },
def preprocess_history_function(examples, tokenizer, max_length=512):
    prompts = []
    
    # 遍历每个样本
    for instruction, input_text, output, history in zip(
        examples['instruction'], 
        examples['input'], 
        examples['output'],
        examples.get('history', [[] for _ in examples['instruction']])  # 如果没有history则使用空列表
    ):
        # 构建历史对话字符串
        history_text = ""
        if history:
            for turn in history:
                # 假设每个turn是一个dict，包含'human'和'assistant'键
                if isinstance(turn, dict):
                    history_text += f"Human: {turn.get('human', '')}\nAssistant: {turn.get('assistant', '')}\n"
                # 或者是一个列表/元组，包含人类和助手的对话
                elif isinstance(turn, (list, tuple)) and len(turn) == 2:
                    history_text += f"Human: {turn[0]}\nAssistant: {turn[1]}\n"
        
        # 构建当前对话的提示语
        if input_text and input_text.strip():
            current_prompt = f"Input: {input_text}\nInstruction: {instruction}\n"
        else:
            current_prompt = f"Instruction: {instruction}\n"
            
        # 组合历史对话和当前对话
        if history_text:
            prompt = f"Chat History:\n{history_text}\nCurrent Dialog:\n{current_prompt}Output: {output}"
        else:
            prompt = f"{current_prompt}Output: {output}"
            
        prompts.append(prompt)

    # Tokenize所有prompts
    tokenized_inputs = tokenizer(
        prompts,
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors="pt"
    )

    # 创建标签
    labels = tokenized_inputs['input_ids'].clone()
    labels[labels == tokenizer.pad_token_id] = -100
    tokenized_inputs['labels'] = labels
    
    return tokenized_inputs

def convert_json_history_to_dataset(json_file_path, save_directory, tokenizer):
    """转换多轮对话JSON数据为Dataset格式"""
    
    # 加载JSON数据
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # 数据清洗和格式化
    cleaned_data = []
    for item in data:
        # 检查必要字段
        if not all(isinstance(item.get(k), str) and len(item.get(k, '').strip()) > 0 
                  for k in ['instruction', 'output']):
            continue
            
        # 确保history字段格式正确
        history = item.get('history', [])
        if not isinstance(history, list):
            history = []
            
        # 验证history中的每个对话回合
        valid_history = []
        for turn in history:
            # 如果是字典格式
            if isinstance(turn, dict) and 'human' in turn and 'assistant' in turn:
                valid_history.append(turn)
            # 如果是列表/元组格式
            elif isinstance(turn, (list, tuple)) and len(turn) == 2:
                valid_history.append({
                    'human': str(turn[0]),
                    'assistant': str(turn[1])
                })
        
        # 创建清洗后的数据项
        cleaned_item = {
            'instruction': item['instruction'],
            'input': item.get('input', ''),
            'output': item['output'],
            'history': valid_history
        }
        cleaned_data.append(cleaned_item)
    
    # 转换为Dataset格式
    dataset = Dataset.from_pandas(pd.DataFrame(cleaned_data))
    
    # 预处理数据
    processed_dataset = dataset.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        remove_columns=dataset.column_names,
        desc="Processing dataset"
    )
    
    # 设置数据格式
    processed_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    # 保存处理后的数据集
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    processed_dataset.save_to_disk(save_directory)
    
    return processed_dataset
```

## 5.3 数据分析

各公司处理业务和使用数据情况不一致，无法全部覆盖主要提供思路

**基础统计信息：**
- 总样本数
- 多轮对话样本数量
- 空输入样本数量
- 平均文本长度

**Token长度分析：**
- 指令、输入、输出的token长度分布
- 总token长度分布
- 各类token长度的统计指标（均值、中位数、最大值等）

**对话深度分析：**
- 对话轮数分布
- 平均对话深度
- 最大对话深度

**内容质量分析：**
- 空值检测
- 特殊字符比例
- 文本质量指标

请大家自行适配环境调整代码：

```python
#  增加了 ...  需要进行代码微调后使用
import json ...
import pandas as pd ...
import numpy as np ...
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns ...
from transformers import AutoTokenizer
from typing import List, Dict, Any
import torch ...
from tqdm import tqdm
import nltk ...
from nltk.tokenize import word_tokenize  
nltk.download('punkt')

class DatasetAnalyzer:
    def __init__(self, json_file_path: str, tokenizer_path: str):
        """
        初始化数据集分析器
        
        参数:
        json_file_path: 训练数据的JSON文件路径
        tokenizer_path: 分词器路径
        """
        self.json_file_path = json_file_path
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)
        self.load_data()
        
    def load_data(self):
        """加载并预处理数据"""
        with open(self.json_file_path, 'r', encoding='utf-8') as f:
            self.raw_data = json.load(f)
        self.df = pd.DataFrame(self.raw_data)
        
    def basic_statistics(self) -> Dict[str, Any]:
        """
        基础统计信息分析
        返回数据集的基本统计信息
        """
        stats = {
            "总样本数": len(self.df),
            "包含历史对话的样本数": sum(self.df['history'].apply(lambda x: len(x) > 0 if isinstance(x, list) else 0)),
            "空输入样本数": sum(self.df['input'].apply(lambda x: not bool(str(x).strip()))),
            "平均指令长度": self.df['instruction'].apply(len).mean(),
            "平均输出长度": self.df['output'].apply(len).mean(),
        }
        
        return stats
    
    def token_length_analysis(self) -> Dict[str, Any]:
        """
        分析token长度分布
        """
        token_stats = {
            "instruction_tokens": [],
            "input_tokens": [],
            "output_tokens": [],
            "total_tokens": []
        }
        
        for _, row in tqdm(self.df.iterrows(), total=len(self.df), desc="分析token长度"):
            # 统计各字段的token长度
            instruction_tokens = len(self.tokenizer.encode(row['instruction']))
            input_tokens = len(self.tokenizer.encode(row['input'])) if row['input'] else 0
            output_tokens = len(self.tokenizer.encode(row['output']))
            total_tokens = instruction_tokens + input_tokens + output_tokens
            
            token_stats["instruction_tokens"].append(instruction_tokens)
            token_stats["input_tokens"].append(input_tokens)
            token_stats["output_tokens"].append(output_tokens)
            token_stats["total_tokens"].append(total_tokens)
        
        # 计算统计信息
        stats = {}
        for key, values in token_stats.items():
            stats[key] = {
                "mean": np.mean(values),
                "median": np.median(values),
                "max": np.max(values),
                "min": np.min(values),
                "95th_percentile": np.percentile(values, 95)
            }
            
        return stats
    
    def dialogue_depth_analysis(self) -> Dict[str, Any]:
        """
        分析对话深度（针对多轮对话）
        """
        dialogue_depths = []
        for history in self.df['history']:
            if isinstance(history, list):
                dialogue_depths.append(len(history))
            else:
                dialogue_depths.append(0)
                
        return {
            "平均对话轮数": np.mean(dialogue_depths),
            "最大对话轮数": np.max(dialogue_depths),
            "对话轮数分布": Counter(dialogue_depths)
        }
    
    def content_quality_analysis(self) -> Dict[str, Any]:
        """
        分析内容质量
        """
        quality_metrics = {
            "空指令数": sum(self.df['instruction'].apply(lambda x: not bool(str(x).strip()))),
            "空输出数": sum(self.df['output'].apply(lambda x: not bool(str(x).strip()))),
            "指令中的特殊字符比例": self.analyze_special_chars('instruction'),
            "输出中的特殊字符比例": self.analyze_special_chars('output'),
        }
        
        return quality_metrics
    
    def analyze_special_chars(self, column: str) -> float:
        """分析特殊字符的比例"""
        special_chars = set('!@#$%^&*()_+={}[]|\\:;"\'<>,.?/~`')
        total_chars = sum(self.df[column].apply(len))
        special_char_count = sum(self.df[column].apply(
            lambda x: sum(1 for c in x if c in special_chars)
        ))
        return special_char_count / total_chars if total_chars > 0 else 0
    
    def plot_token_distribution(self, save_path: str = None):
        """
        绘制token长度分布图
        """
        token_stats = self.token_length_analysis()
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Token Length Distributions')
        
        for (key, values), ax in zip(token_stats.items(), axes.flat):
            sns.histplot(values, ax=ax)
            ax.set_title(f'{key} Distribution')
            ax.set_xlabel('Token Length')
            ax.set_ylabel('Count')
            
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
        plt.show()
    
    def plot_dialogue_depth(self, save_path: str = None):
        """
        绘制对话深度分布图
        """
        depths = self.dialogue_depth_analysis()['对话轮数分布']
        
        plt.figure(figsize=(10, 6))
        plt.bar(depths.keys(), depths.values())
        plt.title('Dialogue Depth Distribution')
        plt.xlabel('Number of Turns')
        plt.ylabel('Count')
        
        if save_path:
            plt.savefig(save_path)
        plt.show()
        
    def generate_report(self, output_path: str = None):
        """
        生成完整的分析报告
        """
        report = {
            "基础统计信息": self.basic_statistics(),
            "Token长度分析": self.token_length_analysis(),
            "对话深度分析": self.dialogue_depth_analysis(),
            "内容质量分析": self.content_quality_analysis()
        }
        
        if output_path:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(report, f, ensure_ascii=False, indent=2)
        
        return report

def main():
    # 使用示例
    analyzer = DatasetAnalyzer(
        json_file_path="path_to_your_data.json",
        tokenizer_path="your_model_path"
    )
    
    # 生成报告
    report = analyzer.generate_report("analysis_report.json")
    
    # 绘制可视化图表
    analyzer.plot_token_distribution("token_distribution.png")
    analyzer.plot_dialogue_depth("dialogue_depth.png")
    
    # 打印关键统计信息
    print("\n=== 数据集分析报告 ===")
    print("\n1. 基础统计信息:")
    for k, v in report["基础统计信息"].items():
        print(f"{k}: {v}")
        
    print("\n2. Token长度分析:")
    for field, stats in report["Token长度分析"].items():
        print(f"\n{field}:")
        for metric, value in stats.items():
            print(f"  {metric}: {value:.2f}")
            
    print("\n3. 对话深度分析:")
    for k, v in report["对话深度分析"].items():
        if k != "对话轮数分布":
            print(f"{k}: {v:.2f}")
            
    print("\n4. 内容质量分析:")
    for k, v in report["内容质量分析"].items():
        print(f"{k}: {v}")

if __name__ == "__main__":
    main()
```

# 6. 总结

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241226162949116.png" width=100%></div>